In [1]:
import tracemalloc

from utils_v1 import *
import traceback


In [2]:
from icecream import ic

import numpy as np
from src.dataset import (
    MultipleDates, PAMultipleDates,
    MTMultipleDates, MSMultipleDates, PIMultipleDates
)

# from src.dataset_legacy import MTDeforestationTime
from src import training_generator as generator
from src.patchesHandler import PatchesHandler, PatchesHandlerMultipleDates, PatchesHandlerEvidential2

from src.manager.multioutput import ManagerMCDropout, ManagerSingleRun, ManagerEvidential2
from mpl_toolkits.axes_grid1 import make_axes_locatable

import cv2
from src import metrics as _metric
import json
import pickle 

from src.Logger import Logger
import pdb
import gc


Tensorflow ver. 2.6.0


In [3]:

with open('config/config.json', 'r') as f:
    config = json.load(f)
    
print("Mode: {}. Site: {}. Training: {}. Training date: {}".format(
    config['mode'], config['site'], config['training'], config['training_date']))
print(config)


Mode: evidential2. Site: MT. Training: True. Training date: current
{'training': True, 'inferring': True, 'site': 'MT', 'training_date': 'current', 'mode': 'evidential2', 'training_times': 10, 'uncertainty_method': 'pred_entropy', 'loadInference': False, 'removePolygons': True, 'plotLandsat': False, 'plotPOI': False, 'training_experiment_id': 60, 'get_multiple_metrics': False, 'start_repetition_id': 0, 'end_repetition_id': 10, 'save_probabilities': False, 'addPastDeforestationInput': True, 'classes_mode': True, 'learning_rate': 0.0001, 'use_cloud_mask': True, 'mask_out_non_biome': False, 'log_filename': 'log.pkl'}


In [4]:

if config['site'] == 'PA':
	if config['training_date'] == 'earlier':
		dates = [2017, 2018]
	else:
		dates = [2018, 2019]
	datasetClass = PAMultipleDates
elif config['site'] == 'MT':
	if config['training_date'] == 'earlier':
		dates = [2018, 2019]
	else:
		dates = [2019, 2020]
	datasetClass = MTMultipleDates
elif config['site'] == 'MS':
	if config['training_date'] == 'earlier':
		dates = [2018, 2019]
	else:
		dates = [2019, 2020]
	datasetClass = MSMultipleDates
elif config['site'] == 'PI':
	if config['training_date'] == 'earlier':
		dates = [2018, 2019]
	else:
		dates = [2019, 2020]
	datasetClass = PIMultipleDates
else:
	raise ValueError('Invalid site')

dataset = datasetClass(config = config,
			   dates = dates, 
	 	addPastDeforestationInput = config['addPastDeforestationInput'],
		borderBuffer = 2)	

if config['mode'] == 'ensemble':
	if config['training'] == True:
		config['mode'] = 'single_run'
	else:
		sys.exit("Ensure that training is True for ensemble mode. Use infer_ensemble.ipynb for inference.")

if config['training'] == False:
	if config['site'] == 'PA':
		if config['mode'] == 'mcd' or config['mode'] == 'single_run': # MCD will do 10 inference on each of the 10 repetitions = 100 runs
			if config['training_date'] == 'current': exp = 0
			elif config['training_date'] == 'earlier': exp = 1
				
		if config['mode'] == 'evidential2':
			if config['training_date'] == 'current': exp = 8; repetition_id = 0
			elif config['training_date'] == 'earlier': 
				# exp = 9; repetition_id = 0
				exp = 30; repetition_id = 0

	elif config['site'] == 'MT':

		if config['mode'] == 'mcd' or config['mode'] == 'single_run':
			if config['training_date'] == 'current': exp = 2
			elif config['training_date'] == 'earlier': exp = 3
		if config['mode'] == 'evidential2':
			if config['training_date'] == 'current': exp = 10; repetition_id = 0
			elif config['training_date'] == 'earlier': exp = 42; repetition_id = 0

	elif config['site'] == 'MS':

		if config['mode'] == 'mcd' or config['mode'] == 'single_run':
			if config['training_date'] == 'current': exp = 40
			elif config['training_date'] == 'earlier': 
				exp = 37

		if config['mode'] == 'evidential2':
			if config['training_date'] == 'current': exp = 12; repetition_id = 0
			elif config['training_date'] == 'earlier': exp = 39; repetition_id = 0
	elif config['site'] == 'PI':

		if config['mode'] == 'mcd' or config['mode'] == 'single_run':
			if config['training_date'] == 'current': 
				exp = 44
				exp = 58
			elif config['training_date'] == 'earlier': exp = 27
		if config['mode'] == 'evidential2':
			if config['training_date'] == 'current': exp = 45; repetition_id = 0
			elif config['training_date'] == 'earlier': exp = 39; repetition_id = 0

else:
	exp = config['training_experiment_id']
ic(dates)


if issubclass(type(dataset), MultipleDates):
	patchesHandler = PatchesHandlerMultipleDates(
		dataset)
elif config['mode'] == 'evidential2':
	patchesHandler = PatchesHandlerEvidential2()
else:
	patchesHandler = PatchesHandler()



ic| list(self.date_ids): [0]
ic| self.image_channels: [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]]
ic| dates: [2019, 2020]
ic| self.input_image_shape: 21
ic| self.dataset.image_channels: [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]]


In [5]:


if config['mode'] == 'evidential2':
    config["inference_times"] = 1
    manager_class = ManagerEvidential2
    config['uncertainty_method'] = "pred_entropy"
    config['classes_mode'] = True
    
if config['mode'] == 'mcd':
    config["inference_times"] = 10
    manager_class = ManagerMCDropout
    config['uncertainty_method'] = "pred_entropy"
    config['classes_mode'] = False

elif config['mode'] == 'single_run':
    config["inference_times"] = 1
    manager_class = ManagerSingleRun
    config['uncertainty_method'] = "pred_entropy_single"
    config['classes_mode'] = False



In [6]:
exp

60

In [7]:
logger = Logger()


In [8]:
if config['training'] == True:
    

    manager = manager_class(config, dataset, patchesHandler, logger)
    print("manager.config['dropout_training']", manager.config['dropout_training'])
    manager.defineExperiment(exp) # fixed
    
    manager.setExperimentPath()
    manager.createLogFolders()

    manager.loadDataset()

    # %%
    if config["training"] == True:
        manager.run()


manager.config['dropout_training'] False
D:/Jorge/datasets/deforestation/MT/deforestation_past_years.tif
Loaded deforestation past years


ic| np.unique(deforestation_past_years, return_counts=True): (array([   0, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017,
                                                                    2018, 2019, 2020, 2021], dtype=uint16),
                                                              array([161124576,   1332784,    240796,    338545,    313901,    287191,
                                                                       708581,    648966,    876898,   1007787,   1098901,   1054092,
                                                                      2096062,   2473828,   1400992], dtype=int64))


Label where deforestation past years is actual date (2020) = 1


ic| np.unique(deforestation_past_years, return_counts=True): (array([   0, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017,
                                                                    2018, 2019, 2020, 2021], dtype=uint16),
                                                              array([161124576,   1332784,    240796,    338545,    313901,    287191,
                                                                       708581,    648966,    876898,   1007787,   1098901,   1054092,
                                                                      2096062,   2473828,   1400992], dtype=int64))


Past deforestation different from 0 (no deforestation)


...

training samples:  (3403, 3) validation samples:  (610, 3)
time:  0


c:\Users\jchamorro\Anaconda3\envs\tf2\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_enc_net (InputLayer)      [(None, 128, 128, 21 0                                            
__________________________________________________________________________________________________
res1_net1 (Conv2D)              (None, 128, 128, 16) 3040        input_enc_net[0][0]              
__________________________________________________________________________________________________
drop_net1 (SpatialDropout2D)    (None, 128, 128, 16) 0           res1_net1[0][0]                  
__________________________________________________________________________________________________
res2_net1 (Conv2D)              (None, 128, 128, 16) 2320        drop_net1[0][0]                  
______________________________________________________________________________________________

c:\Users\jchamorro\Anaconda3\envs\tf2\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


319/319 [==============================] - 28s 89ms/step - loss: 0.0724 - accuracy: 0.6384 - val_loss: 0.0234 - val_accuracy: 0.7310

Epoch 00001: val_loss improved from 0.03399 to 0.02338, saving model to D:/Jorge/datasets/deforestation/experiments/MT/exp60/models\resunet_0.h5
New best val loss. Val loss: 0.02338. Early stop count: 0
Epoch: 3
Anneling Coeficient [0.2]
319/319 [==============================] - 30s 94ms/step - loss: 0.0654 - accuracy: 0.6441 - val_loss: 0.0231 - val_accuracy: 0.7339

Epoch 00001: val_loss improved from 0.02338 to 0.02314, saving model to D:/Jorge/datasets/deforestation/experiments/MT/exp60/models\resunet_0.h5
New best val loss. Val loss: 0.02314. Early stop count: 0
Epoch: 4
Anneling Coeficient [0.3]
319/319 [==============================] - 28s 89ms/step - loss: 0.0617 - accuracy: 0.6478 - val_loss: 0.0199 - val_accuracy: 0.7310

Epoch 00001: val_loss improved from 0.02314 to 0.01985, saving model to D:/Jorge/datasets/deforestation/experiments/MT/exp

In [9]:
print("manager.measured_training_time", manager.measured_training_time)
print("np.sum(np.array(manager.measured_training_time))", np.sum(np.array(manager.measured_training_time)))
print("np.mean(np.array(manager.measured_training_time))", np.mean(np.array(manager.measured_training_time)))

manager.measured_training_time [1627.8607394695282, 2531.797860622406, 1155.0087203979492, 1151.1121423244476, 1103.8306486606598, 1677.816174030304, 1428.035624742508, 1456.7587809562683, 1462.4621908664703, 1464.1914484500885]
np.sum(np.array(manager.measured_training_time)) 15058.87433052063
np.mean(np.array(manager.measured_training_time)) 1505.887433052063


In [10]:
print("manager.measured_training_time", manager.measured_training_time)
print("np.sum(np.array(manager.measured_training_time))", np.sum(np.array(manager.measured_training_time)))
print("np.mean(np.array(manager.measured_training_time))", np.mean(np.array(manager.measured_training_time)))

manager.measured_training_time [1627.8607394695282, 2531.797860622406, 1155.0087203979492, 1151.1121423244476, 1103.8306486606598, 1677.816174030304, 1428.035624742508, 1456.7587809562683, 1462.4621908664703, 1464.1914484500885]
np.sum(np.array(manager.measured_training_time)) 15058.87433052063
np.mean(np.array(manager.measured_training_time)) 1505.887433052063


In [11]:



if config['site'] == 'PA':
	dates = [2018, 2019]
	datasetClass = PAMultipleDates
elif config['site'] == 'MT':
	dates = [2019, 2020]
	datasetClass = MTMultipleDates
elif config['site'] == 'MS':
	dates = [2019, 2020]
	datasetClass = MSMultipleDates
elif config['site'] == 'PI':
	dates = [2019, 2020]
	datasetClass = PIMultipleDates
else:
	raise ValueError('Invalid site')

dataset = datasetClass(config = config,
			   dates = dates, 
	 	addPastDeforestationInput = config['addPastDeforestationInput'],
		borderBuffer = 2)


ic(dates)

if issubclass(type(dataset), MultipleDates):
	patchesHandler = PatchesHandlerMultipleDates(
		dataset)
elif config['mode'] == 'evidential2':
	patchesHandler = PatchesHandlerEvidential2()
else:
	patchesHandler = PatchesHandler()


ic| list(self.date_ids): [0]
ic| self.image_channels: [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]]
ic| dates: [2019, 2020]
ic| self.input_image_shape: 21
ic| self.dataset.image_channels: [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]]


In [12]:
manager_class

src.manager.multioutput.ManagerEvidential2

In [13]:
exp

60

In [14]:
repetition_ids = manager_class(config, dataset, patchesHandler, logger).get_repetition_ids(exp)  
repetition_n = len(repetition_ids)
print("repetition_ids {}, repetition_n {}".format(repetition_ids, repetition_n))


repetition_ids [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], repetition_n 10


In [15]:
config['start_repetition_id']

0

In [16]:
if config['inferring'] == True:
    results = []
    error_count = 0

    # while idx < len(repetition_n):
    # for idx in range(config['start_repetition_id'], repetition_n): # 10 repetitions
    for idx in range(config['start_repetition_id'], config['end_repetition_id']): # 10 repetitions

    # for idx in range(0, 4): # 10 repetitions


        print("Beginning run number {}".format(idx))
        logger = Logger()
        manager = manager_class(config, dataset, patchesHandler, logger)

        print("manager.config", manager.config)

        manager.defineExperiment(exp) # fixed
        manager.defineRepetitionId(idx) # varying from 0 to 10
        manager.setExperimentPath()
        manager.createLogFolders()

        manager.loadDataset()

        # %%
        if config['get_multiple_metrics'] == False:
            predictor_fcn = manager.run_predictor
        else:
            predictor_fcn = manager.run_predictor_multiple_metrics
        result = predictor_fcn()
        print("Result idx {}: {}".format(idx, result))
        results.append(result)

        with open(manager.default_log_name, 'wb') as f:
            pickle.dump(results, f)
        print("Grid execution idx: {}".format(idx))
        del manager
        gc.collect()
        # idx += 1
    

Beginning run number 0


manager.config {'training': True, 'inferring': True, 'site': 'MT', 'training_date': 'current', 'mode': 'evidential2', 'training_times': 10, 'uncertainty_method': 'pred_entropy', 'loadInference': False, 'removePolygons': True, 'plotLandsat': False, 'plotPOI': False, 'training_experiment_id': 60, 'get_multiple_metrics': False, 'start_repetition_id': 0, 'end_repetition_id': 10, 'save_probabilities': False, 'addPastDeforestationInput': True, 'classes_mode': True, 'learning_rate': 0.0001, 'use_cloud_mask': True, 'mask_out_non_biome': False, 'log_filename': 'log.pkl', 'inference_times': 1, 'dropout_training': False}
D:/Jorge/datasets/deforestation/MT/deforestation_past_years.tif
Loaded deforestation past years


ic| np.unique(deforestation_past_years, return_counts=True): (array([   0, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017,
                                                                    2018, 2019, 2020, 2021], dtype=uint16),
                                                              array([161124576,   1332784,    240796,    338545,    313901,    287191,
                                                                       708581,    648966,    876898,   1007787,   1098901,   1054092,
                                                                      2096062,   2473828,   1400992], dtype=int64))


Label where deforestation past years is actual date (2020) = 1


ic| np.unique(deforestation_past_years, return_counts=True): (array([   0, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017,
                                                                    2018, 2019, 2020, 2021], dtype=uint16),
                                                              array([161124576,   1332784,    240796,    338545,    313901,    287191,
                                                                       708581,    648966,    876898,   1007787,   1098901,   1054092,
                                                                      2096062,   2473828,   1400992], dtype=int64))


...

[0.74321114 0.90453497 0.67707459 0.02999291 0.5274697  0.57821123
 0.12686563]
threshold:  0.19665064319003717
threshold 0.19665064319003717


ic| label_current_deforestation_test_high_uncertainty.shape: (1337931,)
    predicted_test_high_uncertainty.shape: (1337931,)
ic| TP_H + FN_H + FP_H + TN_H: 1337931
    len(label_mask_current_deforestation_test): 44599801
ic| TP_H + FN_H + FP_H + TN_H: 1337931
    len(label_mask_current_deforestation_test): 44599801


[0.74321535 0.90454481 0.67705247 0.02999859 0.52748311 0.57822424
 0.12685966]
threshold 0.19665064319003717
(44599801,) (44599801,)


ic| self.m_optimal: {'AA': array([0.02999859]),
                     'UEO': array([0.12685966]),
                     'f1': 75.23,
                     'f1_H': array([0.55168941]),
                     'f1_L': array([0.81598233]),
                     'precision_H': array([0.52748311]),
                     'precision_L': array([0.74321535]),
                     'recall_H': array([0.57822424]),
                     'recall_L': array([0.90454481]),
                     'recall_Ltotal': array([0.67705247])}
ic| self.m_audited_optimal: {'f1': array([0.85878539]),
                             'precision': array([0.79877014]),
                             'recall': array([0.92855172])}


cm_audited [[43539947   200925]
 [   61369   797560]]
[0.79877014 0.92855172]
Result idx 9: {'uncertainty_result': {'metrics': {'precision_L': array([0.74321535]), 'recall_L': array([0.90454481]), 'recall_Ltotal': array([0.67705247]), 'AA': array([0.02999859]), 'precision_H': array([0.52748311]), 'recall_H': array([0.57822424]), 'UEO': array([0.12685966]), 'f1_L': array([0.81598233]), 'f1_H': array([0.55168941]), 'f1': 75.23}, 'metrics_audited': {'precision': array([0.79877014]), 'recall': array([0.92855172]), 'f1': array([0.85878539])}, 'exp': 60}}
Grid execution idx: 9


In [17]:
error_count

0

In [18]:
print(results)

[{'uncertainty_result': {'metrics': {'precision_L': array([0.76362707]), 'recall_L': array([0.86546587]), 'recall_Ltotal': array([0.62511487]), 'AA': array([0.02999827]), 'precision_H': array([0.48547673]), 'recall_H': array([0.58477541]), 'UEO': array([0.15528666]), 'f1_L': array([0.81136336]), 'f1_H': array([0.53051957]), 'f1': 73.15}, 'metrics_audited': {'precision': array([0.82350326]), 'recall': array([0.90282773]), 'f1': array([0.86134302])}, 'exp': 60}}, {'uncertainty_result': {'metrics': {'precision_L': array([0.87167724]), 'recall_L': array([0.91223392]), 'recall_Ltotal': array([0.69490475]), 'AA': array([0.03000444]), 'precision_H': array([0.62719469]), 'recall_H': array([0.63002441]), 'UEO': array([0.10276496]), 'f1_L': array([0.89149456]), 'f1_H': array([0.62860637]), 'f1': 82.98}, 'metrics_audited': {'precision': array([0.90120221]), 'recall': array([0.93314318]), 'f1': array([0.9168946])}, 'exp': 60}}, {'uncertainty_result': {'metrics': {'precision_L': array([0.75186296])

In [19]:
print(len(results))

10


In [20]:
results[7]

{'uncertainty_result': {'metrics': {'precision_L': array([0.76776975]),
   'recall_L': array([0.9441362]),
   'recall_Ltotal': array([0.68871142]),
   'AA': array([0.02999713]),
   'precision_H': array([0.57172493]),
   'recall_H': array([0.68123937]),
   'UEO': array([0.12408454]),
   'f1_L': array([0.84686804]),
   'f1_H': array([0.62169613]),
   'f1': 78.67},
  'metrics_audited': {'precision': array([0.82158006]),
   'recall': array([0.95924949]),
   'f1': array([0.88509341])},
  'exp': 60}}